<a href="https://colab.research.google.com/github/flapperz/music-playlist-transfer/blob/main/joox_2_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get Joox music list

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
#@title Joox URL
src_url = 'https://www.joox.com/th/playlist/USERPLAYLIST_0_23536932_100000029_1618063434?appshare=null' #@param {type:"string"}

In [ ]:
page = requests.get(src_url)
soup = BeautifulSoup(page.content, 'html.parser')
html_body = list(soup.children)[1]


In [ ]:
html_song_names = list(html_body.select('b.SongName a'))
html_song_authors = list(html_body.select('div.SongDesc'))

In [ ]:
song_names = [tag.get_text() for tag in html_song_names]
song_authors = [[childtag.get_text() for childtag in tag.children] for tag in html_song_authors]
print(song_names[:5])
print(song_authors[:5])

['Another One Bites the Dust', 'The Most Beautiful Thing', 'Why iii Love The Moon.', "Livin' Thing", 'Yellow Hearts']
[['Queen'], ['Bruno Major'], ['Phony PPL'], ['The Beautiful South'], ['Ant Saunders']]


## Add song to youtube music

In [ ]:
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

In [ ]:
client_secret_path = "client_secret_777709478932-jb8cm20botr1s8qblngmdpo2iq45karf.apps.googleusercontent.com.json" #@param {type:"string"}

In [ ]:
api_service_name = "youtube"
api_version = "v3"
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

# Get credentials and create an API client
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
    client_secret_path, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=777709478932-jb8cm20botr1s8qblngmdpo2iq45karf.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=pBxZVeXcm9sbObWDxfCt5fxSvkriJX&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g7VnRamDcUlalnlXJHzXvwyzlX7GiVXdvz2WVfSgiIRVloliuncKnc


In [ ]:
#@title Name of Playlist
title = '' #@param {type: 'string'}
request = youtube.playlists().insert(
        part="snippet,status",
        body={
          "snippet": {
            "title": title,
            "description": "Create with music playlist transfer by Krit Cholapand",
            "defaultLanguage": "en"
          },
          "status": {
            "privacyStatus": "private"
          }
        }
    )
response = request.execute()
playlist_id = response['id']

In [ ]:
search_snippets = []
for i in range(len(song_names)):
    name = song_names[i]
    authors = song_authors[i]
    search_string = f"{name} {' '.join(authors)}"
    print(search_string)
    try:
        request = youtube.search().list(part='snippet',type='video', topicId='/m/04rlf', q=search_string, videoCategoryId='10')
        snippet = request.execute()
        time.sleep(1)
    except Exception as e:
        print(e)
    search_snippets.append(snippet)

Another One Bites the Dust Queen
The Most Beautiful Thing Bruno Major
Why iii Love The Moon. Phony PPL
Livin' Thing The Beautiful South
Yellow Hearts Ant Saunders
Why'd You Only Call Me When You're High? Arctic Monkeys
Piledriver waltz Alex Turner
I Don't Trust Myself (With Loving You) John Mayer
Watermelon Sugar Harry Styles
Lyin' Eyes The Eagles
Cornerstone Arctic Monkeys
Golden Days Whitney
Time After Time (Album Version) Cyndi Lauper
Should I Stay or Should I Go (Remastered) The Clash
Take on Me A-Ha
Happy Together The Turtles
Hold On, We're Going Home Drake
Come And Get Your Love Redbone
Say You Like Me We The Kings
Keeping A Secret Bleachers
Rollercoaster Bleachers
The Oogum Boogum Song Brenton Wood
Easy Commodores
Stand by Me Oasis
Don't Look Back in Anger Oasis
Stop Crying Your Heart Out Oasis
Young And Beautiful Lana Del Rey
Suit & Tie Justin Timberlake Jay-Z
The Man Who Can't Be Moved The Script
Sweet Disposition The Temper Trap
Kiss Prince & The Revolution
Heroes David Bowie

In [ ]:
for snippet in search_snippets:
    items = snippet['items']
    if items:
        video_id = items[0]['id']['videoId']
        try:
            request = youtube.playlistItems().insert(
            part="snippet",
            body={
            "snippet": {
                "playlistId": playlist_id,
                "position": 0,
                "resourceId": {
                "kind": "youtube#video",
                "videoId": video_id
                }
            }
            }
            )
        
            response = request.execute()
        except Exception as e:
            print(e)
        time.sleep(1)
    else:
        print('Search not matched')

<HttpError 500 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=snippet&alt=json returned "Internal error encountered.". Details: "Internal error encountered.">
